__Нейросетевая языковая модель на основе CharCNN+LSTM__ 

Евгений Борисов <esborisov@sevsu.ru>

---

In [1]:
# загружаем текст
import gzip
# with gzip.open('../data/dostoevsky-besy-p2.txt.gz','rt',encoding='utf-8') as f: data = f.read()     
with gzip.open('../data/lobas-taxisty.txt.gz','rt',encoding='utf-8') as f: data = f.read()     
print(len(data))

601744


In [2]:
print(data[:512])

Желтые короли. Записки нью-йоркского таксиста.

В.Лобас



Товарищам моим - белым и черным, американцам и эмигрантам из России и Израиля, из Греции и Кореи, арабам, китайцам, полякам и всем прочим таксистам города Нью-Йорк в знак глубокого уважения к их нечеловеческому труду эту горькую книгу посвящаю...

Водитель No 320718


"Я никогда не знал бы многое из того что я знаю, и половины чего достаточно, чтобы отравить навсегда несколько человеческих жизней, если бы мне не пришлось сделаться шофером такси..." 


In [3]:
# print(data[:1024*100])

In [ ]:
from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word
import itertools

In [ ]:
class TextData:
    
    def __init__(self,text,language='russian'):
        self._word2index = dict()
        self._index2word = dict()
        
        self._max_word_len = 0
        self._max_sentence_len = 0
        
        self._data = self._tokenize(text)
        self._check_len()
        self._build_vocab()
        
    @staticmethod
    def _tokenize(text):
        return [ # выполняем токенизацию
            nltk_tokenize_word(s,language=language) # разбиваем предложения на слова
            for s in nltk_sentence_split(text,language=language) # режем текст на отдельные предложения
        ]
    
    def _check_len(self):
        self._max_word_len = max([ len(t) for t in set(itertools.chain(*self._data)) ])
        self._max_sentence_len = max([ len(s) for s in self._data ])
        return self
    
    def _build_vocab(self): # формируем словарь 
        # список токенов за исключением специальных
        words = sorted( set(itertools.chain(*self._data)) ) 
        # нумеруем токены
        self._word2index = { w:i for i,w in enumerate(words) }
        # обратное преобразование из номера в токен
        self._index2word = { i:w for  w,i in self._word2index.items() }

        return self

In [ ]:
class TextCharData(TextData):
    
    def __init__(self,text,language='russian'):
        super().__init__(text,language)
        self._abc2index = dict()
        self._index2abc = dict()
        self._build_abc()
        
    def _build_abc(self): # формируем словарь 
        abc = sorted(set(' '.join(set(itertools.chain(*self._data)))))
        self._abc2index  = { w:i for i,w in enumerate(abc) }
        self._index2abc = { i:w for  w,i in self._abc2index.items() }
        return self

In [ ]:
class Sequencer:
    
    

In [7]:
# from scipy.sparse
# sentence = text[6]
# len(sentence)*len(abc2index)
# max_sentence_len*len(abc2index)
# abc2index

---

In [8]:
from scipy import sparse
import numpy as np

def token2map(token,abc2index=abc2index,max_word_len=max_word_len):
    return sparse.csr_matrix( 
        tuple([ 
            (1,)*len(token), # индикатор (значение в ячейке матрицы)
            (   # индексы матрицы
                tuple(range(len(token))), # позиция символа в слове
                tuple([ abc2index[c] for c in token ]), # номер символа в словаре
            ) 
        ]), 
        shape=[max_word_len,len(abc2index)],
    )

token2map(token='дополняем').shape

NameError: name 'abc2index' is not defined

In [45]:
def sentence2map(
        sentence,
        abcabc2index=abc2index,
        max_word_len=max_word_len,
        max_sentence_len=max_sentence_len,
    ):
        return [ # дополняем до максиамального размера предложения нулями
            sparse.csr_matrix(np.zeros( ( max_word_len,len(abc2index) ) ) )  
            for _ in range(max_sentence_len-len(sentence)) 
        ] + [ # кодируем предложение
            token2map(token,abc2index=abc2index,max_word_len=max_word_len) # строим карту токена
            for token in sentence 
        ]

# len( sentence2map(sentence) )
# 198,(25,139)

In [46]:
# sentence = text[6]

In [ ]:
# sequences = np.array([
#     [ word2index[token] for token in sentence[i:i+seq_len] ] # собираем токены и их левые контексты
#     for sentence in text if len(sentence)>seq_len # все предложения, которые длиннее размера контекста  
#     for i in range(len(sentence)-seq_len) # двигаем окно размера seq_len по предложению
# ])


In [19]:
# # собираем датасет [ левый контекст, слово ]
# import numpy as np
# import keras.utils as ku 

# seq_len = 16+1 # размер левого контекста + 1 слово

# sequences = np.array([
#     [ word2index[token] for token in sentence[i:i+seq_len] ] # собираем токены и их левые контексты
#     for sentence in text if len(sentence)>seq_len # все предложения, которые длиннее размера контекста  
#     for i in range(len(sentence)-seq_len) # двигаем окно размера seq_len по предложению
# ])

# # sequences.shape

# inputs = sequences[:,:-1] # контексты
# targets = sequences[:,-1] # целевые токены для контекстов 

# del sequences

# # выполняем OHE, конвертируем номера токенов targets в векторы {0,1}
# targets = ku.np_utils.to_categorical(targets, num_classes=total_words)

# inputs.shape, inputs.dtype, targets.shape, targets.dtype

((41529, 16), dtype('int64'), (41529, 24319), dtype('float32'))

In [ ]:
def sentence2map(
        sentence,
        abc_len=len(abc2index),
        max_word_len=max_word_len,
        max_sentence_len=max_sentence_len

In [ ]:
def batch_generator(inputs,targets,,batch_size=32):
    for i in range(0,len(text),batch_size):
        
        

In [ ]:
# собираем языковую модель

from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Dense
    
model = Sequential()
model.add(Embedding(total_words, 128, input_length=inputs.shape[1]))
model.add(LSTM(1024))
# model.add(LSTM(128,return_sequences=True))
# model.add(LSTM(128))
model.add(Dense(targets.shape[1], activation='softmax'))

model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
model.summary() # описание нейросети

In [ ]:
%%time 

# обучем модель по левому контексту предсказывать следующий за ним токен
history = model.fit( 
    inputs,
    targets, 
    epochs=10, 
    verbose=1,
)

In [ ]:
from matplotlib import pyplot as plt

# изменение значений ф-ции потери и погрешности в процессе обучения модели

loss = { k:history.history[k] for k in history.history if k.find('loss')>-1 }  
accuracy = { k:history.history[k] for k in history.history if k.find('acc')>-1 }  

fig,(ax1,ax2) = plt.subplots(1,2,figsize=(16,3))

for k in loss: 
    ax1.plot(loss[k], label=k)
    ax1.legend()    
    ax1.grid()
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    
    
for k in accuracy: 
    ax2.plot(accuracy[k], label=k)
    ax2.legend()    
    ax2.grid()
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')

    
plt.show()

In [ ]:
# тестируем модель

from numpy import random as rng
from random import sample
from keras.preprocessing.sequence import pad_sequences

# выбираем случайный контекст из исходных данных
def get_sample(text,seq_len=seq_len-2,max_seq_len=seq_len-1):
    assert seq_len<max_seq_len
    for _ in range(100):
        sentence = sample(text,1)[0]
        if len(sentence)>seq_len*2:
            i = rng.randint(len(sentence)-seq_len*2)
            return sentence[i:i+seq_len]
    return [PAD,]

# кодируем контекст
def encode_sentence(sentence,word2index=word2index,max_seq_len=seq_len-1):
    return np.array(
            pad_sequences(
                [[ word2index[token] for token in sentence ],], 
                maxlen=max_seq_len, 
                padding='pre',
            )
        )

# encode_sentence(get_sample(text)).shape

In [ ]:
for __ in range(10):

    sentence0 = get_sample(text) # выбираем контекст
    answer = [] # предсказанные токены
    sentence = [PAD]+sentence0 # начальный контекст

    for _ in range(27):
        sequence = encode_sentence(sentence[1:]) # кодируем контекст
        predicted_index = np.argmax( model.predict(sequence) ) # продолжение контекста то модели 
        predicted_word = index2word[predicted_index] # декодируем ответ модели
        answer.append(predicted_word) # запоминаем токен предсказанный моделью
        if predicted_index == word2index[EOS]: break # если предсказан конец предложения то завершаем
        sentence.append(predicted_word) # предсказанный токен в контекст
        
    print('\n'+'init> '+' '.join(sentence0)+'\n'+'lstm> ' + ' '.join(answer) )
    print(' - '*30 )    

In [4]:
# # выполняем токенизацию
# import itertools
# from nltk.tokenize import sent_tokenize as nltk_sentence_split
# from nltk.tokenize import word_tokenize as nltk_tokenize_word

# text = [ 
#     nltk_tokenize_word(s,language='russian') # разбиваем предложения на слова
#     for s in nltk_sentence_split(data,language='russian') # режем текст на отдельные предложения
# ]

# # del data

# max_word_len = max([ len(t) for t in set(itertools.chain(*text)) ])
# max_sentence_len = max([ len(s) for s in text ])

# len(text), max_sentence_len, max_word_len

In [5]:
# abc = sorted(set(' '.join(set(itertools.chain(*text)))))
# # abc =  sorted( set(data) - set(['\n',]) )
# abc2index  = { w:i for i,w in enumerate(abc) }
# index2abc = { i:w for  w,i in abc2index.items() }
# del abc

# len(abc2index)

In [6]:
# # формируем словарь 

# # список токенов за исключением специальных
# words = sorted( set(itertools.chain(*text)) ) 

# # нумеруем токены
# word2index = { w:i for i,w in enumerate(words) }
# del words

# # обратное преобразование из номера в токен
# index2word = { i:w for  w,i in word2index.items() }

# total_words = len(word2index) # всего токенов в словаре

# total_words

In [9]:
# abc2index

In [7]:
# from scipy.sparse

# sentence = text[6]

# len(sentence)*len(abc2index)

# max_sentence_len*len(abc2index)